In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql
import numpy as np

where_map = {}
# pairs regexp: "\[([a-zA-z0-9$\-+τ\.%]*)\]-", "-\[([a-zA-z0-9$\-+τ]*)\]",  or ( c in ['τ'] for c in s)
ignorable_files = ["farm_risk_yield_data_2021_06_21.csv", "Farm_risk_06_21.csv", "yield_data_2021_06_21.csv","Alternate.csv","vfat_tools_data_2021_06_21.csv",'.ipynb_checkpoints', "Manual Farm Yield 6:21.xlsx", "crash_parser.ipynb", "yieldFarm_6-21.csv"]


In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [3]:
def get_raydium_data(date):
    sql = f'select * from raydium_farm_pool_data where timestamp_dt = FROM_UNIXTIME({date});'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate']
    data['network'] = 'SOL'
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

def get_liquidityfolio_pool_data(date):
    sql = f'select * from liquidityfolio_pool where timestamp_dt = FROM_UNIXTIME({date}) and `where` != "balancer";'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate'] * 100 * 12
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    data['network'] = 'ETH'
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

In [4]:
date = datetime(2021, 6, 21)
dt = date.timestamp()

In [5]:
df = get_raydium_data(dt)
df = df.append(get_liquidityfolio_pool_data(dt))
df[df['where'] == 'Sushi']

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ETH,UMA,4.298114,3797.635432,Sushi,ETH,POOL
1,ETH,UWL,3.876399,52.328631,Sushi,ETH,POOL
4,ETH,LIGHT,0.139979,1.981946,Sushi,ETH,POOL
9,DGCL,ACXT,2.414292,12.988530,Sushi,ETH,POOL
11,UNI,xSUSHI,5.109741,0.011464,Sushi,ETH,POOL
...,...,...,...,...,...,...,...
1443,ETH,AMALGAM,0.087070,0.172533,Sushi,ETH,POOL
1444,USDP,DUCK,50.641893,0.000000,Sushi,ETH,POOL
1445,DAI,RDM,0.000000,1.513494,Sushi,ETH,POOL
1450,ETH,89SA,5.153667,3.998762,Sushi,ETH,POOL


In [6]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
}


In [7]:
dex_map = {
    "SUSHI" : "https://app.sushi.com"
    
}

In [8]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        if all(ord(c) < 128 for c in s):
            return s
        else:
            print(line)
            return None
    except:
        return ''

def pair2(line):
#     print(line)
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
#     print(s)
    if all(ord(c) < 128 for c in s):
        return s
    else:
        print(line)
        return ''

In [9]:
def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [10]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

In [11]:
def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").replace('*', '').strip()
    return None

In [12]:
def get_single_pair(line):
    exp = "\d* *.*Price:"
    x = re.findall(exp, line)
    if not len(x) or "TVL:" in line:
#         print(">>>>>>>>>>>",line)
        return
    pair = x[0].replace(" Price:", "")
    return pair[pair.rindex(" "):].strip() if " " in pair else pair

def get_single_tvl(line):
    exp = "\([0-9\,\.$]+\)"
    x = re.findall(exp, line)
    return x[0].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()


In [13]:
tvl("[WBTC]-[WAVAX] PGL [+] [-] [<=>] Price: $160,706,376.77 TVL: $12,218,373.64\n")

'12218373.64'

In [14]:
def is_end_of_block(line):
#     print(line, line == "" or len(re.findall("INFO[ ]+: ", line)))
    return line.strip() == "" or len(re.findall("INFO[ ]+: ", line)) or "******" in line

def read_block(lines, start_index):
    block = []
#     print("Block Start Line No.: ", start_index + 1)
    while start_index < len(lines):
        line = lines[start_index]
        start_index = start_index + 1
        if not is_end_of_block(line):
            block.append(line.strip())
        else:
            break
    return [block, start_index]

def read_blocks_from_files(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    start_index = 0
    while start_index < len(lines):
        line = lines[start_index]
        if not is_end_of_block(lines[start_index]):
            block_and_index = read_block(lines, start_index)
#             print(block_and_index[0])
            if block_and_index is not None:
                block_start_line_no.append(start_index + 1)
                start_index = block_and_index[1]
                blocks.append(block_and_index[0])
            else:
                start_index = start_index + 1
        elif len(re.findall("INFO[ ]+: ", line)):
            block_start_line_no.append(start_index + 1)
            blocks.append([line])
            start_index = start_index + 1
        else:
            start_index = start_index + 1
    return [blocks, block_start_line_no]

def read_blocks_from_files_as(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    for line in lines:
        line_no = line_no + 1
        line = line.strip()
        if line == "" or len(re.findall("INFO[ ]+: ", line)):
            blocks.append(block)
            block_start_line_no.append(line_no)
            if len(line):
                blocks.append([line])
                block_start_line_no.append(line_no + 1)
            block = []
        else:
            block.append(line)
            
    blocks.append(block)
    return [blocks, block_start_line_no]

def get_pair_info_from_block(block, wh, network):
    tvl_index = 0
    if network == 'BSC':
        tvl_index = 3
    if len(block) == 1:
        if 'INFO' in block[0]:
            return [where(block[0])]
        else:
            return [wh] if len(wh) else None
    if len(block) and pair1(block[0]) and not get_single_pair(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [pair1(block[0]), pair2(block[0]), get_single_tvl(block[tvl_index]) if tvl_index > 0 else tvl(block[tvl_index]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_single_pair(block[0]):
#         print('singel')
        if network == 'BSC':
            tvl_index = 1
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [get_single_pair(block[0]), "", get_single_tvl(block[1]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    return None


def get_pairs(blocks, network, block_start_line_no):
    where = ""
    pairs = []
    index = -1
    for block in blocks:
#         print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        index = index + 1
        try:
            pair = get_pair_info_from_block(block, where, network)
            if pair == None:
                print('no Pair....\n',block)
                print("Block Start Line No.: ", block_start_line_no[index] if index < len(block_start_line_no) else -1)
#             if len(where) == 0 and pair and len(pair) > 2:
#                 print(block, network)
            if pair is not None and len(pair) > 0:
                if len(pair) == 1 and pair[0] and len(pair[0]):
                    where = pair[0]
                else:
                    pairs.append(pair)
        except Exception as e:
            print("Exception", traceback2.format_exc())
            print(block[0])
    return pairs
        

In [15]:
block = [['334 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,875,027,502.08 TVL: $15,943,098.17', 'τBTC Price: $26,860.23', 'BTCB Price: $32,310.00', 'Staked: 0.0008 Cake-LP ($1,494,384.52)', 'AUTO Per Week: 3.22 ($3,410.84)', 'APR: Day 0.03% Week 0.23% Year 11.87%', 'You are staking 0.00 [τBTC]-[BTCB] Cake LP ($0.00), 0.00% of the pool.', 'Stake 0.00 [τBTC]-[BTCB] Cake LP', 'Unstake 0.00 [τBTC]-[BTCB] Cake LP', 'Claim 0.00 AUTO ($0.00)', 'Staking or unstaking also claims rewards.']]
get_pairs(block, "AVAX", [1])

334 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,875,027,502.08 TVL: $15,943,098.17
no Pair....
 ['334 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,875,027,502.08 TVL: $15,943,098.17', 'τBTC Price: $26,860.23', 'BTCB Price: $32,310.00', 'Staked: 0.0008 Cake-LP ($1,494,384.52)', 'AUTO Per Week: 3.22 ($3,410.84)', 'APR: Day 0.03% Week 0.23% Year 11.87%', 'You are staking 0.00 [τBTC]-[BTCB] Cake LP ($0.00), 0.00% of the pool.', 'Stake 0.00 [τBTC]-[BTCB] Cake LP', 'Unstake 0.00 [τBTC]-[BTCB] Cake LP', 'Claim 0.00 AUTO ($0.00)', 'Staking or unstaking also claims rewards.']
Block Start Line No.:  1


[]

In [16]:
data_dir = './'
def conver_files_to_one_df(data_dir):
    all_pairs = []
    files = os.listdir(data_dir)
    for file_name in files:
        if file_name in ignorable_files:
            continue
        network = file_name[0:file_name.index(" ")]
        if 'Alternate' == network:
            network = 'ETH'
#         if 'MATIC' != network:
#             continue
        print(file_name)
        blocks = read_blocks_from_files(f"{data_dir}{file_name}")
        block_start_line_no = blocks[1]
        blocks = blocks[0]
        pairs = get_pairs(blocks, network, block_start_line_no)
        all_pairs.extend(pairs)
    return all_pairs

In [17]:
pairs = conver_files_to_one_df(data_dir)

AVAX 6:21 Yields.crash
33 - [WAVAX]-[🐆] Uni LP [+] [-] [<=>] Price: $29.07 TVL: $997.27
MATIC 6:21 Yields.crash
no Pair....
 ['Total Staked: $1,405,756.61', 'Found 13 pools.']
Block Start Line No.:  3242
Exception Traceback (most recent call last):
  File "<ipython-input-14-33b151aaa9ee>", line 104, in get_pairs
  File "<ipython-input-14-33b151aaa9ee>", line 82, in get_pair_info_from_block
  File "<ipython-input-8-6396f5945b4a>", line 19, in pair2
ValueError: substring not found

GFI-USDC Quick LP [+] [-] [<=>] Price: $178,320.46 TVL: $588,100.42
Exception Traceback (most recent call last):
  File "<ipython-input-14-33b151aaa9ee>", line 104, in get_pairs
  File "<ipython-input-14-33b151aaa9ee>", line 82, in get_pair_info_from_block
  File "<ipython-input-8-6396f5945b4a>", line 19, in pair2
ValueError: substring not found

GFI-WETH Quick LP [+] [-] [<=>] Price: $8.04 TVL: $636,153.45
0 - [🌾]-[WETH] SLP [+] [-] [<=>] Price: $1.56 TVL: $463.56
no Pair....
 ['0 - [🌾]-[WETH] SLP [+] [-] [<=

no Pair....
 ['Staking Contract', 'Found 416 pools.']
Block Start Line No.:  14
361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,875,027,502.08 TVL: $15,943,098.17
no Pair....
 ['361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,875,027,502.08 TVL: $15,943,098.17', 'τBTC Price: $26,860.23', 'BTCB Price: $32,310.00', 'Staked: 0.0084 Cake-LP ($15,663,079.16)', 'CAKE Per Week: 10887.59 ($149,704.32)', 'APR: Day 0.14% Week 0.96% Year 49.70%', 'You are staking 0.00 [τBTC]-[BTCB] Cake LP ($0.00), 0.00% of the pool.', 'Stake 0.00 [τBTC]-[BTCB] Cake LP', 'Unstake 0.00 [τBTC]-[BTCB] Cake LP', 'Claim 0.00 CAKE ($0.00)', 'Staking or unstaking also claims rewards.']
Block Start Line No.:  1179
380 - [WBNB]-[τDOGE] Cake LP [+] [-] [<=>] Price: $1,897,678.70 TVL: $3,470,242.35
no Pair....
 ['Staking Contract', 'Found 78 pools.']
Block Start Line No.:  1829
334 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $1,875,027,502.08 TVL: $15,943,098.17
no Pair....
 ['334 - [τBTC]-[BTCB] Cake LP [+] [-] [<

 ['Staking Contract', 'Found 12 pools.']
Block Start Line No.:  49302
no Pair....
 ['Staking Contract', 'Found 36 pools.']
Block Start Line No.:  49467
no Pair....
 ['Staking Contract', 'Found 4 pools.']
Block Start Line No.:  49792
no Pair....
 ['Staking Contract', 'Found 22 pools.']
Block Start Line No.:  49857
no Pair....
 ['Staking Contract', 'Found 46 pools.']
Block Start Line No.:  50116
no Pair....
 ['Staking Contract', 'Found 5 pools.']
Block Start Line No.:  50643
no Pair....
 ['Total Staked: $2,274.30', 'Staking Contract', 'Found 2 pools.']
Block Start Line No.:  50708
no Pair....
 ['Staking Contract', 'Found 24 pools.']
Block Start Line No.:  50750
no Pair....
 ['Staking Contract', 'Found 13 pools.']
Block Start Line No.:  50971
no Pair....
 ['Staking Contract', 'Found 14 pools.']
Block Start Line No.:  51132
no Pair....
 ['Staking Contract', 'Found 12 pools.']
Block Start Line No.:  51303
no Pair....
 ['Staking Contract', 'Found 17 pools.']
Block Start Line No.:  51444
no P

In [18]:
pair_df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'tvl', 'apr', 'where', 'network'])
pair_df['apy'] = np.nan

In [19]:
pairs

[['WAVAX',
  'ETH',
  '51087305.47',
  '15.51',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['WBTC',
  'WAVAX',
  '9929223.81',
  '13.76',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['WAVAX',
  'LINK',
  '11323389.21',
  '13.69',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['PNG',
  'WAVAX',
  '47499806.35',
  '138.39',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['WAVAX',
  'USDT',
  '4164438.97',
  '14.23',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['SUSHI',
  'WAVAX',
  '5165806.16',
  '13.68',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['WAVAX',
  'DAI',
  '4644283.81',
  '13.97',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['AAVE',
  'WAVAX',
  '2157007.18',
  '13.82',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['WAVAX',
  'UNI',
  '965436.29',
  '13.93',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 ['YFI',
  'WAVAX',
  '342228.15',
  '13.84',
  'https://app.pangolin.exchange/#/png',
  'AVAX'],
 [

In [20]:
pair_df = pair_df.drop(pair_df[pair_df['pair1'] == ''].index)

In [21]:
pair_df[pair_df['pair1'] == '']

,pair1,pair2,tvl,apr,where,network,apy


In [22]:
pair_df[pair_df['network'] == 'Alternate']['network'] = 'ETH'

In [23]:
sheet_pairs_df = pd.DataFrame([])
eth_sheet_pairs_df = pd.read_csv('Alternate.csv')
# eth_sheet_pairs_df.columns=['pair1', 'pair2', 'tvl', 'where', 'apr']
eth_sheet_pairs_df['apy'] = np.nan
eth_sheet_pairs_df['network'] = 'ETH'
sheet_pairs_df = sheet_pairs_df.append(eth_sheet_pairs_df)
sheet_pairs_df['pair2'] = sheet_pairs_df['pair2'].fillna('')

In [24]:
sheet_pairs_df

,pair1,pair2,tvl,where,apr,network,apy
0,WETH,USDT,239639702,SUSHI,12.41,ETH,NaN
1,USDC,WETH,327505750,SUSHI,11.35,ETH,NaN
2,DAI,WETH,"194,526,818",SUSHI,16.95,ETH,NaN
3,sUSD,WETH,"9,631,753",SUSHI,16.19,ETH,NaN
4,COMP,WETH,"19,628,399",SUSHI,9.04,ETH,NaN
...,...,...,...,...,...,...,...
200,BOND,WETH,"85,991",SUSHI,69.73,ETH,NaN
201,bb_cDAI,DAI,"183,726",SUSHI,32.64,ETH,NaN
202,WETH,FOLD,"241,785",SUSHI,24.80,ETH,NaN
203,NDX,WETH,"152,541",SUSHI,39.31,ETH,NaN


In [25]:
def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [26]:
def get_symbol_mapping(pair):
    if pair is not None:
        return sm[pair.strip()] if pair.strip() in sm else pair.strip()
    else:
        return ""

In [27]:
pair_df = pair_df.append(sheet_pairs_df).reset_index(drop=True)
pair_df['type_of_agriculture'] = 'FARM'
pair_df = pair_df.append(df).reset_index(drop=True)

In [28]:

pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: get_symbol_mapping(x))

In [29]:
pair_df['pair2'].unique()

array(['ETH', 'AVAX', 'LINK', ..., 'CHZ', 'POLS', '89SA'], dtype=object)

In [30]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,AVAX,ETH,51087305.47,15.51,https://app.pangolin.exchange/#/png,AVAX,NaN,FARM
1,BTC,AVAX,9929223.81,13.76,https://app.pangolin.exchange/#/png,AVAX,NaN,FARM
2,AVAX,LINK,11323389.21,13.69,https://app.pangolin.exchange/#/png,AVAX,NaN,FARM
3,PNG,AVAX,47499806.35,138.39,https://app.pangolin.exchange/#/png,AVAX,NaN,FARM
4,AVAX,USDT,4164438.97,14.23,https://app.pangolin.exchange/#/png,AVAX,NaN,FARM
...,...,...,...,...,...,...,...,...
9386,ETH,0xMR,5.316245,6.74223696,Uniswap,ETH,NaN,POOL
9387,ETH,CHZ,42.029085,45.54486288,Uniswap,ETH,NaN,POOL
9388,ETH,POLS,5510.79352,14.9398974,Uniswap,ETH,NaN,POOL
9389,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [31]:
# pair_df['apr'].unique().tolist()

In [32]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if pd.isnull( apr):
            return
        if apr is not None and len(apr.strip()):
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if pd.isnull( apy):
            return
        if apy is not None and len(apy.strip()):
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(9391, 8)
(55, 8)


In [33]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
880,ETH,,0.00,20292961517981806592.00,https://www.polyion.net,MATIC,NaN,FARM
882,BTC,,0.00,123656475.50,https://www.polyion.net,MATIC,NaN,FARM
886,QUICK,,0.00,76882494111164039168.00,https://www.polyion.net,MATIC,NaN,FARM
936,USDC,POLLEN,4.83,1832032.34,https://polybutterfly.finance,MATIC,NaN,FARM
942,USDC,,0.00,292502730.36,https://polybutterfly.finance,MATIC,NaN,FARM
...,...,...,...,...,...,...,...,...
9386,ETH,0xMR,5.316245,6.74223696,Uniswap,ETH,NaN,POOL
9387,ETH,CHZ,42.029085,45.54486288,Uniswap,ETH,NaN,POOL
9388,ETH,POLS,5510.79352,14.9398974,Uniswap,ETH,NaN,POOL
9389,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [34]:
pair_df[pair_df['apr'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
7679,REDUCE,,0,NaN,SUSHI,ETH,NaN,FARM
7714,SAFFRONCHEF,,0,NaN,SUSHI,ETH,NaN,FARM
7771,kmWETH,USDT-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7772,kmWETH,USDC-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7773,kmWETH,DAI-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7774,kmWBTC,USDT-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7775,kmWBTC,DAI-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7776,kmWBTC,USDC-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7777,kmLINK,USDT-LINK,0,NaN,SUSHI,ETH,NaN,FARM
7778,kmLINK,DAI-LINK,0,NaN,SUSHI,ETH,NaN,FARM


In [35]:
pair_df = pair_df.drop_duplicates()

In [36]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [37]:
single_piar_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
46,sBAMBOO-V2,,0.00,0,https://pandaswap.exchange,AVAX,0.000000,FARM
59,BURN,,0.00,0,https://pandaswap.exchange,AVAX,0.000000,FARM
85,OLIVE,,112053.57,89.88,https://avax.olive.cash,AVAX,145.394070,FARM
107,AVAX,,997.27,201.98,https://avax.olive.cash,AVAX,649.496940,FARM
109,LYD,,452084.35,342.07,https://www.lydia.finance,AVAX,2910.738297,FARM
...,...,...,...,...,...,...,...,...
7679,REDUCE,,0,NaN,SUSHI,ETH,NaN,FARM
7714,SAFFRONCHEF,,0,NaN,SUSHI,ETH,NaN,FARM
7830,MASTERCHEFDUMMY,,0.00,NaN,SUSHI,ETH,NaN,FARM
8335,Xi,,0.0,0.0,Sushi,ETH,NaN,POOL


In [38]:
double_coin_pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,AVAX,ETH,51087305.47,15.51,https://app.pangolin.exchange/#/png,AVAX,16.773626,FARM
1,BTC,AVAX,9929223.81,13.76,https://app.pangolin.exchange/#/png,AVAX,14.748670,FARM
2,AVAX,LINK,11323389.21,13.69,https://app.pangolin.exchange/#/png,AVAX,14.668404,FARM
3,PNG,AVAX,47499806.35,138.39,https://app.pangolin.exchange/#/png,AVAX,298.000502,FARM
4,AVAX,USDT,4164438.97,14.23,https://app.pangolin.exchange/#/png,AVAX,15.289050,FARM
...,...,...,...,...,...,...,...,...
9386,ETH,0xMR,5.316245,6.74223696,Uniswap,ETH,NaN,POOL
9387,ETH,CHZ,42.029085,45.54486288,Uniswap,ETH,NaN,POOL
9388,ETH,POLS,5510.79352,14.9398974,Uniswap,ETH,NaN,POOL
9389,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [39]:
tmp_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,AVAX,ETH,51087305.47,15.51,https://app.pangolin.exchange/#/png,AVAX,16.773626,FARM
1,BTC,AVAX,9929223.81,13.76,https://app.pangolin.exchange/#/png,AVAX,14.748670,FARM
2,AVAX,LINK,11323389.21,13.69,https://app.pangolin.exchange/#/png,AVAX,14.668404,FARM
3,PNG,AVAX,47499806.35,138.39,https://app.pangolin.exchange/#/png,AVAX,298.000502,FARM
4,AVAX,USDT,4164438.97,14.23,https://app.pangolin.exchange/#/png,AVAX,15.289050,FARM
...,...,...,...,...,...,...,...,...
9386,ETH,0xMR,5.316245,6.74223696,Uniswap,ETH,NaN,POOL
9387,ETH,CHZ,42.029085,45.54486288,Uniswap,ETH,NaN,POOL
9388,ETH,POLS,5510.79352,14.9398974,Uniswap,ETH,NaN,POOL
9389,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [40]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [41]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,NaN,POOL
1,,xSUSHI,0.0,0.0,Sushi,ETH,NaN,POOL
2,$BASED,ETH,13.959925,11.399762160000002,Uniswap,ETH,NaN,POOL
3,$BASED,SUSD,15.820221,7.164307559999999,Uniswap,ETH,NaN,POOL
4,$DG,BNB,2303008.23,676.04,https://pancakeswap.finance,BSC,81023.324312,FARM
...,...,...,...,...,...,...,...,...
13099,zLOT,ETH,20.127711,1.3924665600000001,Sushi,ETH,NaN,POOL
13100,zSUSHI,ZERO,79387.12,0.00,https://app.0.exchange,AVAX,0.000000,FARM
13101,zUNI,ZERO,12502.69,0.00,https://app.0.exchange,AVAX,0.000000,FARM
13102,zUSDC,ZERO,63160.84,0.00,https://app.0.exchange,AVAX,0.000000,FARM


In [42]:
summary = {
    "total_pairs": pair_df.shape[0],
"single_coin_pairs": single_piar_df.shape[0],
"double_coin_piars": double_coin_pair_df.shape[0],
"after_switching_coins_total_double_coin_pairs": double_coin_pair_df.shape[0] * 2,
"total_pairs_in_sheet": double_coin_pair_df.shape[0] * 2 + single_piar_df.shape[0]
}

In [43]:
summary

{'total_pairs': 9084,
 'single_coin_pairs': 2532,
 'double_coin_piars': 6552,
 'after_switching_coins_total_double_coin_pairs': 13104,
 'total_pairs_in_sheet': 15636}

In [44]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

In [45]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,NaN,POOL
1,,xSUSHI,0.0,0.0,Sushi,ETH,NaN,POOL
2,$BASED,ETH,13.959925,11.399762160000002,Uniswap,ETH,NaN,POOL
3,$BASED,SUSD,15.820221,7.164307559999999,Uniswap,ETH,NaN,POOL
4,$DG,BNB,2303008.23,676.04,https://pancakeswap.finance,BSC,8.102332e+04,FARM
...,...,...,...,...,...,...,...,...
15631,zs-XUSD2,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000e+00,FARM
15632,zs-tsBTC,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000e+00,FARM
15633,τBTC,,187.91,244.00,https://www.elixir-swap.io,BSC,1.038026e+03,FARM
15634,🌾,,33.96,1087.75,https://harvester.app,MATIC,4.518613e+06,FARM


In [46]:
pair_df['dex'] = pair_df['where']
pair_df['date'] = date 
del pair_df['where']

In [47]:
# pair_df.to_csv('yield_data_2021_06_21.csv')

In [48]:
pair_df['dex'].unique().tolist()

['Sushi',
 'Uniswap',
 'https://pancakeswap.finance',
 'https://quickswap.exchange/#/quick',
 'https://strudel.finance',
 'https://swap.cometh.io/#/stake',
 'https://www.bao.finance',
 'https://caramelswap.finance',
 'https://absorber.finance',
 'https://icecreamswap.finance',
 'http://mangofarm.finance',
 'https://brussels.worldswap.finance',
 'https://1inch.exchange/#/dao/farming',
 'SUSHI',
 'https://www.convexfinance.com',
 'https://app.swamp.finance',
 'https://ellipsis.finance',
 'https://frozenyogurt.finance',
 'https://nerve.fi',
 'https://autofarm.network',
 'https://blue.planetfinance.io',
 'https://glacierswap.org/farms/',
 'https://8ballfinance.io',
 'https://dollar.supra.finance',
 'https://polyvolve.finance',
 'https://polygaj.finance',
 'https://www.polycake.finance',
 'https://polyrangers.finance',
 'https://polyvalent.finance',
 'https://polyfi.io',
 'https://polylion.exchange',
 'https://www.orcinuslabs.ca',
 'https://polywolf.finance',
 'https://www.polygoat.finance'

In [49]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
6542,ETH,,3012.78,457.77,MATIC,9355.974306,FARM,https://polyfox.finance,2021-06-21
6543,ETH,,1372.17,0.00,MATIC,0.000000,FARM,https://hawkdex.com,2021-06-21
6544,ETH,,0.12,0.00,MATIC,0.000000,FARM,https://farm.BoughtThe.top,2021-06-21
6545,ETH,,596.66,61.64,MATIC,85.128521,FARM,https://www.polystarter.fi,2021-06-21
6546,ETH,,126.97,305.33,MATIC,1991.779088,FARM,https://polyvolve.finance,2021-06-21
...,...,...,...,...,...,...,...,...,...
8720,ETH,zHEGIC,2.145807,0.81045276,ETH,NaN,POOL,Uniswap,2021-06-21
8721,ETH,zHEGIC,0.019139,0.0,ETH,NaN,POOL,Sushi,2021-06-21
8722,ETH,zLOT,0.011083,1.08214668,ETH,NaN,POOL,Uniswap,2021-06-21
8723,ETH,zLOT,20.127711,1.3924665600000001,ETH,NaN,POOL,Sushi,2021-06-21


In [50]:
pair_df['apr'].unique().tolist()

['0.0',
 '11.399762160000002',
 '7.164307559999999',
 '676.04',
 '81.48',
 '48.312018480000006',
 '87.70',
 '2.6847416400000004',
 '144.92',
 '3.7541240399999998',
 '1.96526784',
 '0.00',
 '17.5935342',
 '5.68',
 '5.831882759999999',
 '6.74223696',
 '0.12823259999999997',
 '0.8739114',
 '0',
 '286.05',
 '440.49',
 '50.14',
 '17.45',
 '0.15',
 '17.665750080000002',
 '2.11926708',
 '0.40851059999999995',
 nan,
 '0.9120171600000001',
 '0.38475768000000005',
 '1.43477616',
 '1.15140624',
 '0.6166921200000001',
 '0.0035435999999999996',
 '35.95',
 '3.55',
 '19.40',
 '10.76',
 '33.94',
 '4.25',
 '0.08788151999999999',
 '172.60',
 '0.07637052',
 '5.15366652',
 '2.40',
 '1.880382',
 '237.01',
 '92.55',
 '325.47',
 '160.50',
 '149.29',
 '18.78',
 '291.49',
 '407.43',
 '331.32',
 '317.94',
 '109.80',
 '21.92',
 '100.55',
 '230.97',
 '127.32',
 '23.28',
 '171.17',
 '513.51',
 '1369.36',
 '454.24',
 '79619885192114814976.00',
 '361.69',
 '74.30',
 '703.14',
 '1378.03',
 '268.34',
 '2830634.32',
 '

In [51]:
pair_df

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
0,,sil,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-21
1,,xSUSHI,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-21
2,$BASED,ETH,13.959925,11.399762160000002,ETH,NaN,POOL,Uniswap,2021-06-21
3,$BASED,SUSD,15.820221,7.164307559999999,ETH,NaN,POOL,Uniswap,2021-06-21
4,$DG,BNB,2303008.23,676.04,BSC,8.102332e+04,FARM,https://pancakeswap.finance,2021-06-21
...,...,...,...,...,...,...,...,...,...
15631,zs-XUSD2,,0.00,0,ETH,0.000000e+00,FARM,https://www.stabilize.finance/app/,2021-06-21
15632,zs-tsBTC,,0.00,0,ETH,0.000000e+00,FARM,https://www.stabilize.finance/app/,2021-06-21
15633,τBTC,,187.91,244.00,BSC,1.038026e+03,FARM,https://www.elixir-swap.io,2021-06-21
15634,🌾,,33.96,1087.75,MATIC,4.518613e+06,FARM,https://harvester.app,2021-06-21


In [52]:
pair_df[pair_df['pair1'] == 'ADA']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
149,ADA,,8438.54,0.00,BSC,0.000000,FARM,https://www.goosedefi.com,2021-06-21
150,ADA,,131694.87,0,BSC,0.000000,FARM,https://www.goosedefi.com,2021-06-21
151,ADA,,2205.03,0.00,BSC,0.000000,FARM,http://slime.finance,2021-06-21
152,ADA,,267066.09,21.07,BSC,23.446689,FARM,https://koaladefi.finance/,2021-06-21
153,ADA,,3156.58,338.56,BSC,2807.837183,FARM,https://brickchain.finance,2021-06-21
...,...,...,...,...,...,...,...,...,...
259,ADA,USDT,70720.38,21.02,BSC,23.385017,FARM,https://app.swamp.finance,2021-06-21
260,ADA,USDT,312.75,104.61,BSC,184.227214,FARM,https://frankenstein.finance,2021-06-21
261,ADA,USDT,12059572.47,48.35,BSC,62.122177,FARM,https://mdex.com,2021-06-21
262,ADA,dADA,2987.53,0.00,BSC,0.000000,FARM,https://iron.finance/,2021-06-21


In [53]:
df['pair1'].unique().tolist()

['ALEPH',
 'APYS',
 'BOLE',
 'BOP',
 'BTC',
 'CCAI',
 'CHAD',
 'CHEEMS',
 'COPE',
 'ETH',
 'FIDA',
 'FROG',
 'FTR',
 'FTT',
 'KEEP',
 'KERMIT',
 'KIN',
 'LINK',
 'LUA',
 'MAPS',
 'MEDIA',
 'MER',
 'MOLA',
 'OXY',
 'RAY',
 'ROPE',
 'SAMO',
 'SBFC',
 'SDOGE',
 'SHBL',
 'SKEM',
 'SLIM',
 'SOL',
 'SOLAPE',
 'SOLPAD',
 'SRM',
 'STEP',
 'SUSHI',
 'TOMO',
 'TULIP',
 'TYNA',
 'unknown',
 'USDC',
 'USDT',
 'WOOF',
 'xCOPE',
 'YFI',
 'WBTC',
 'WHALE',
 'FRONT',
 'DGCL',
 'UNI',
 'YUNO',
 'DAI',
 'MIS',
 'QDAO',
 'BAO',
 'REN',
 'DATA',
 'crDAI',
 'sUSD',
 'CRO',
 'MIS3',
 'AXIE',
 'cDAI',
 'xSUSHI',
 'BNS',
 'SASHIMI',
 'MKR',
 'silv2',
 'FORTH',
 'WAN',
 'KP3R',
 'VXC',
 'OOKS',
 'ALBT',
 'MIC',
 'KITTY',
 'jEUR',
 'sil',
 'EASY',
 'KAIJU',
 'LEND',
 'PROS',
 'MASK',
 'FOMC',
 'BNT',
 'OHM',
 'ARTH',
 'ANT',
 'DIA',
 'yveCRV',
 'CURA',
 'RUNE',
 'ibETH',
 'WOLF',
 'Xi',
 'SAKE',
 'RAI',
 'TUSD',
 'NAMI',
 'oneLINK',
 'XOR',
 'HEGIC',
 'FARM',
 'HBTC',
 'BUSD',
 'wCOINBASE',
 'Mars',
 'plDai',
 

## Farm Risk

In [54]:
# # farms_risks_df = pd.read_excel("~/notebooks/vfat_tool_data/Farm_risk.xlsx", sheet_name='BSC')
# farm_risk_df = pd.read_csv('~/notebooks/vfat_tool_data/farm_risks_analysis.csv')

In [55]:
# farm_risk_df.columns = ['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', 'rugdoc_url', 'network']
# farm_risk_df


In [56]:
# pairs_farm_risk_df = pd.merge(pair_df, farm_risk_df, how='left', left_on=['dex', 'network'], right_on=['Farm Url', 'network'])
# pairs_farm_risk_df.to_csv('farm_risk_yield_data_2021_06_14.csv')
# pairs_farm_risk_df

In [57]:
# farms_df = pair_df[['dex', 'network']].drop_duplicates().reset_index(drop=True)

In [58]:
# farms_df

In [59]:
# new_farm_risk_df = pd.merge(farms_df, farm_risk_df, how='left', left_on=['dex', 'network'], right_on=['Farm Url', 'network'])
# new_farm_risk_df['Farm Url'] = new_farm_risk_df['dex']
# del new_farm_risk_df['dex']
# new_farm_risk_df = new_farm_risk_df[['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', 'rugdoc_url', 'network']]

# # new_farm_risk_df.columns = ['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', 'rugdoc_url', 'network']


In [60]:
# new_farm_risk_df.to_csv('Farm_risk_06_14.csv', index=False)

In [61]:
excel_file = '~/notebooks/vfat_tool_data/Farm_risk_20210616.xlsx'

In [62]:
bsc_risks_df = pd.read_excel(excel_file, sheet_name='BSC')
bsc_risks_df.columns = ['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2', '3']
bsc_risks_df['rugdoc_url'] = bsc_risks_df['1'].fillna('') + bsc_risks_df['2'].fillna('') + bsc_risks_df['3'].fillna('')
del bsc_risks_df['1']
del bsc_risks_df['2']
del bsc_risks_df['3']
bsc_risks_df['network'] = 'BSC'
bsc_risks_df


,#,Indiv. Assesment,Audit Score,RugDoc,Farm Name,Farm Symbol,Farm Url,rugdoc_url,network
0,x,NaN,NaN,least,Pancake,CAKE,https://pancakeswap.finance,https://rugdoc.io/project/pancakeswap/,BSC
1,NaN,NaN,NaN,some,Apeswap,BANANA,https://apeswap.finance,https://rugdoc.io/project/apeswap/,BSC
2,NaN,NaN,NaN,NaN,Autofarm,AUTOFARM,https://autofarm.network,,BSC
3,x,9.0,NaN,NaN,Bake,BAKE,https://www.bakeryswap.org,,BSC
4,x,NaN,86.0,NaN,Brew,BREW,https://cafeswap.finance,https://www.certik.org/projects/cafeswap,BSC
...,...,...,...,...,...,...,...,...,...
324,NaN,NaN,NaN,low,VenomSwap,VENOM,https://www.venomswap.net,https://rugdoc.io/project/venom-swap/,BSC
325,NaN,NaN,NaN,low,Elephant,ELEPHANT,https://app.elephantswap.finance,https://rugdoc.io/project/elephant-swap/,BSC
326,No TLV,NaN,NaN,NaN,Bloodmoon,BLOODMOON,https://bloodmoon.werewolf.farm,,BSC
327,NaN,NaN,NaN,low,Blacksheep,BLACKSHEEP,https://www.blacksheepdefi.com,https://rugdoc.io/project/blacksheep-finance/,BSC


In [63]:
matic_risks_df = pd.read_excel(excel_file, sheet_name='MATIC')
matic_risks_df.columns = ['#', 'Indiv. Assesment', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2']
matic_risks_df['Audit Score'] = np.nan
matic_risks_df['rugdoc_url'] = matic_risks_df['1'].fillna('') + matic_risks_df['2'].fillna('')
del matic_risks_df['1']
del matic_risks_df['2']
matic_risks_df['network'] = 'MATIC'
matic_risks_df

,#,Indiv. Assesment,RugDoc,Farm Name,Farm Symbol,Farm Url,Audit Score,rugdoc_url,network
0,x,low,NaN,Quickswap *,QUICK,https://quickswap.exchange/#/quick,NaN,,MATIC
1,x,some,NaN,Cometh,MUST,https://swap.cometh.io/#/stake,NaN,,MATIC
2,No TLV,NaN,NaN,Dark,DB,https://www.dark.build,NaN,,MATIC
3,NaN,NaN,NaN,Smartdex,NIOX,https://swap.smartdex.app,NaN,,MATIC
4,x,high,NaN,Elk,ELK,https://app.elk.finance,NaN,,MATIC
...,...,...,...,...,...,...,...,...,...
85,No TLV,NaN,NaN,PolyZero,ZERO,NaN,NaN,,MATIC
86,NaN,NaN,low,PolyMax,MAX,https://polymax.club,NaN,https://rugdoc.io/project/polymax/,MATIC
87,NaN,NaN,low,PolySnowFarm,SNOW,https://polysnow.farm,NaN,https://rugdoc.io/project/polysnow-farm/,MATIC
88,NaN,NaN,low,Polyyeld,YELD,https://polyyeld.finance,NaN,https://rugdoc.io/project/polyyeld-finance/,MATIC


In [64]:
avax_risks_df = pd.read_excel(excel_file, sheet_name='AVAX')
avax_risks_df.columns = ['#', 'Indiv. Assesment', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2']
avax_risks_df['Audit Score'] = np.nan
avax_risks_df['rugdoc_url'] = avax_risks_df['1'].fillna('') + avax_risks_df['2'].fillna('')
del avax_risks_df['1']
del avax_risks_df['2']
avax_risks_df['network'] = 'AVAX'
avax_risks_df


,#,Indiv. Assesment,RugDoc,Farm Name,Farm Symbol,Farm Url,Audit Score,rugdoc_url,network
0,x,low,NaN,Pangolin,PNG,https://app.pangolin.exchange/#/png,NaN,,AVAX
1,x,low,NaN,Complus,COM,https://avax.complus.exchange,NaN,,AVAX
2,NaN,NaN,NaN,Zero,ZERO,https://app.0.exchange,NaN,,AVAX
3,NaN,NaN,NaN,Yeti,YTS,https://exchange.yetiswap.app,NaN,,AVAX
4,x,NaN,low,Pandaswap,BAMBOO,https://pandaswap.exchange,NaN,https://rugdoc.io/project/panda-swap-defi/,AVAX
5,x,some,NaN,Pefi,PEFI,https://penguinfinance.org,NaN,,AVAX
6,x,high,NaN,Elk *,ELK,https://elk.finance,NaN,,AVAX
7,No TLV,NaN,NaN,Snowball,SNOB,https://snowball.network/earn,NaN,,AVAX
8,x,some,NaN,Olive,OLIVE,https://avax.olive.cash,NaN,,AVAX
9,x,NaN,some,Lydia,LYD,https://www.lydia.finance,NaN,https://rugdoc.io/project/lydia-avax/,AVAX


In [65]:
fantom_risks_df = pd.read_excel(excel_file, sheet_name='FANTOM')
fantom_risks_df.columns = ['#', 'Indiv. Assesment', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2']
fantom_risks_df['Audit Score'] = np.nan
fantom_risks_df['rugdoc_url'] = fantom_risks_df['1'].fillna('') + fantom_risks_df['2'].fillna('')
del fantom_risks_df['1']
del fantom_risks_df['2']
fantom_risks_df['network'] = 'FANTOM'
fantom_risks_df


,#,Indiv. Assesment,RugDoc,Farm Name,Farm Symbol,Farm Url,Audit Score,rugdoc_url,network
0,NaN,NaN,NaN,Popsicle,ICE,https://popsicle.finance,NaN,,FANTOM
1,x,NaN,low,Spooky,boo,https://spookyswap.finance,NaN,https://rugdoc.io/project/spookyswap/,FANTOM
2,x,NaN,some,Spirit,SPIRIT,https://app.spiritswap.finance,NaN,https://rugdoc.io/project/spiritswap/,FANTOM
3,x,NaN,low,Borg,SON,https://ftm.borgswap.exchange,NaN,https://rugdoc.io/project/borgswap-kling/,FANTOM
4,x,NaN,low,Hyperjump,ORI,https://hyperjump.fi,NaN,https://rugdoc.io/project/hyperjump-fi/,FANTOM
5,x,NaN,low,Ester,EST,https://ester.finance,NaN,https://rugdoc.io/project/ester-finance/,FANTOM
6,x,high,NaN,Froyo,FROYO,https://frozenyogurt.finance,NaN,,FANTOM
7,NaN,NaN,NaN,Ghost,GHOST,https://www.theghost.finance,NaN,,FANTOM
8,No TLV,NaN,low,Beefy,Various,https://fantom.beefy.finance,NaN,https://rugdoc.io/project/beefy-finance/,FANTOM
9,x,some,NaN,Liquid Driver,LQDR,https://www.liquiddriver.finance,NaN,,FANTOM


In [66]:
risk_df = bsc_risks_df.append(matic_risks_df)
risk_df = risk_df.append(avax_risks_df)
risk_df = risk_df.append(fantom_risks_df)
risk_df = risk_df[risk_df['rugdoc_url'] != '']
risk_df

,#,Indiv. Assesment,Audit Score,RugDoc,Farm Name,Farm Symbol,Farm Url,rugdoc_url,network
0,x,NaN,NaN,least,Pancake,CAKE,https://pancakeswap.finance,https://rugdoc.io/project/pancakeswap/,BSC
1,NaN,NaN,NaN,some,Apeswap,BANANA,https://apeswap.finance,https://rugdoc.io/project/apeswap/,BSC
4,x,NaN,86.0,NaN,Brew,BREW,https://cafeswap.finance,https://www.certik.org/projects/cafeswap,BSC
5,NaN,NaN,NaN,some,Donut,DONUT,https://donutdefi.com,https://rugdoc.io/project/donutswapp/,BSC
7,NaN,NaN,NaN,NaN,Goose,GOOSE,https://www.goosedefi.com,https://www.certik.org/projects/ramenswap,BSC
...,...,...,...,...,...,...,...,...,...
17,xx,NaN,NaN,high,Paintswap,BRUSH,https://paintswap.finance,https://rugdoc.io/project/paintswap/,FANTOM
18,xx,NaN,NaN,low,Tomb,Various,https://tomb.finance,https://rugdoc.io/project/tomb-finance/,FANTOM
19,NaN,NaN,NaN,high,Shadow,Various,https://shadowswap.app,https://rugdoc.io/project/shadowswap-finance/,FANTOM
20,NaN,NaN,NaN,low,Borgswap,KLING,https://klingftm.borgswap.exchange,https://rugdoc.io/project/borgswap-kling/,FANTOM


In [67]:
pairs_farm_risk_df = pd.merge(pair_df, risk_df, how='left', left_on=['dex', 'network'], right_on=['Farm Url', 'network'])
pairs_farm_risk_df.to_csv('farm_risk_yield_data_2021_06_21.csv')
pairs_farm_risk_df

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date,#,Indiv. Assesment,Audit Score,RugDoc,Farm Name,Farm Symbol,Farm Url,rugdoc_url
0,,sil,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,xSUSHI,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$BASED,ETH,13.959925,11.399762160000002,ETH,NaN,POOL,Uniswap,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$BASED,SUSD,15.820221,7.164307559999999,ETH,NaN,POOL,Uniswap,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$DG,BNB,2303008.23,676.04,BSC,8.102332e+04,FARM,https://pancakeswap.finance,2021-06-21,x,NaN,NaN,least,Pancake,CAKE,https://pancakeswap.finance,https://rugdoc.io/project/pancakeswap/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15631,zs-XUSD2,,0.00,0,ETH,0.000000e+00,FARM,https://www.stabilize.finance/app/,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15632,zs-tsBTC,,0.00,0,ETH,0.000000e+00,FARM,https://www.stabilize.finance/app/,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15633,τBTC,,187.91,244.00,BSC,1.038026e+03,FARM,https://www.elixir-swap.io,2021-06-21,NaN,NaN,NaN,high,Elixir,ELXR,https://www.elixir-swap.io,https://rugdoc.io/project/elixir-swap/
15634,🌾,,33.96,1087.75,MATIC,4.518613e+06,FARM,https://harvester.app,2021-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
